In [ ]:
import numpy as np
import pylandstats as pls
import os
import matplotlib.pyplot as plt
import rasterio
from pathlib import Path  # 处理文件路径
import pandas as pd
import rasterio as rio
from rasterio.plot import show
import numpy as np
import matplotlib.pyplot as plt
from osgeo import gdal

res = np.asarray([30, 30])


def patch_fragment_habitat_change(lc_pre, lc_post, landform, center_x, center_y):
    res = np.asarray([30, 30])

    lc_pre[(lc_pre == 1) | (lc_pre == 8)] = 1
    lc_pre[lc_pre > 1] = 2
    lc_pre[lc_pre < 1] = 0

    lc_post[(lc_post == 1) | (lc_post == 8)] = 1
    lc_post[lc_post > 1] = 2
    lc_post[lc_post<1]=0

    natural_mask_2000 = lc_pre
    natural_mask_2000[(natural_mask_2000==2)&(landform==2)]=3
    ls00 = pls.Landscape(natural_mask_2000, res)
    custom_subset_class_metrics_df00 = ls00.compute_class_metrics_df(
        metrics=[ "patch_density",  "largest_patch_index","area_mn"],)

    re00 = pd.DataFrame(custom_subset_class_metrics_df00)
    re00.to_csv("change_land_metrics_00_" + str(center_x) + '_' + str(center_y) + ".csv", index=True)

    natural_mask_2020 =natural_mask_2000
    natural_mask_2020[lc_post ==1]=1
    natural_mask_2020[(natural_mask_2020 == 2) & (landform == 2)] = 3
    ls20 = pls.Landscape(natural_mask_2020, res)
    custom_subset_class_metrics_df00 = ls20.compute_class_metrics_df(
        metrics=["patch_density", "largest_patch_index", "area_mn"], )

    re20 = pd.DataFrame(custom_subset_class_metrics_df00)
    re20.to_csv("change_land_metrics_20_" + str(center_x) + '_' + str(center_y) + ".csv",
        index=True)

def process_dataset(dataset_path1, dataset_path2, dataset_path3, blocksize):
    dataset1 = gdal.Open(dataset_path1)
    width = dataset1.RasterXSize
    height = dataset1.RasterYSize

    nodata_value1 = dataset1.GetRasterBand(1).GetNoDataValue()

    dataset2 = gdal.Open(dataset_path2)
    nodata_value2 = dataset2.GetRasterBand(1).GetNoDataValue()

    dataset3 = gdal.Open(dataset_path3)
    nodata_value3 = dataset3.GetRasterBand(1).GetNoDataValue()
    geotransform = dataset3.GetGeoTransform()
    cols = dataset2.RasterXSize  
    rows = dataset2.RasterYSize  
    projection = dataset1.GetProjection()

    total_blocks_x = width // blocksize
    total_blocks_y = height // blocksize

    for i in range(0, height, blocksize):  # lat-->row

        for j in range(0, width, blocksize):  # lon-->col

            x_size = min(blocksize, width - j)
            y_size = min(blocksize, height - i)

            # 读取数据块
            data_block1 = dataset1.ReadAsArray(j, i, x_size, y_size)
            data_block1[data_block1 == nodata_value1] = 0
            data_block_lc00 = data_block1 // 10

            data_block2 = dataset2.ReadAsArray(j, i, x_size, y_size)
            data_block2[data_block2 == nodata_value2] = 0
            data_block2[data_block2 >= 2] = 2

            data_block3 = dataset3.ReadAsArray(j, i, x_size, y_size)
            data_block3[data_block3 == nodata_value3] = 0
            data_block_lc20 = data_block3 // 10
            human = (data_block_lc20 == 1) | (data_block_lc20 == 8) | (data_block_lc00 == 1) | (data_block_lc00 == 8)

            if np.max(human) > 0:
               
                block_x = j // blocksize
                block_y = i // blocksize
              
                center_x = geotransform[0] + (j + blocksize / 2) * geotransform[1]
                center_y = geotransform[3] + (i + blocksize / 2) * geotransform[5]
                print(j, i, '------', center_x, center_y)
                patch_fragment_habitat_change (data_block_lc00, data_block_lc20, data_block2, center_x, center_y)

    dataset = None
 

if __name__ == "__main__":

    dataset_path1 = "GLCV00.tif"
    dataset_path2 = r"GBLU.tif"
    dataset_path3 = r"GLCV20.tif"
    process_dataset(dataset_path1, dataset_path2, dataset_path3, blocksize=167)